In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import logging
import sys

import ferc1_transform_refactor as tfr
import pandas as pd

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter("%(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
dataset_params = tfr.DatasetTransformParams(**tfr.TRANSFORM_PARAMS)
dataset_params

DatasetTransformParams(__root__={'test_table_id': {'rename_columns': {'netgen': 'net_generation_kwh', 'heat_content': 'fuel_btu_per_unit', 'units': 'fuel_units', 'fuel_type': 'fuel_type'}, 'unit_conversions': {'net_generation_kwh': {'multiplier': 0.001, 'pattern': '(.*)_kwh$', 'repl': '\\1_mwh'}, 'fuel_btu_per_unit': {'multiplier': 1e-06, 'pattern': '(.*)_btu(.*)$', 'repl': '\\1_mmbtu\\2'}}, 'string_categories': {'fuel_type': {'coal': {'xenoforming', 'dinosaur', 'black', 'goo'}, 'oil': {'sunflower', 'astroglide', 'fish', 'olive'}, 'gas': {'cow farts', 'unicorn farts', 'elon musk'}}}}})

In [5]:
table_params = tfr.TableTransformParams(**dataset_params.test_table_id)
table_params

TableTransformParams(rename_columns=RenameCols(__root__={'netgen': 'net_generation_kwh', 'heat_content': 'fuel_btu_per_unit', 'units': 'fuel_units', 'fuel_type': 'fuel_type'}), unit_conversions=MultiColumnUnitConversion(__root__={'net_generation_kwh': UnitConversion(multiplier=0.001, adder=0.0, pattern=re.compile('(.*)_kwh$'), repl='\\1_mwh'), 'fuel_btu_per_unit': UnitConversion(multiplier=1e-06, adder=0.0, pattern=re.compile('(.*)_btu(.*)$'), repl='\\1_mmbtu\\2')}), string_categories=MultiColumnStringCategories(__root__={'fuel_type': StringCategories(__root__={'coal': {'xenoforming', 'dinosaur', 'black', 'goo'}, 'oil': {'sunflower', 'olive', 'astroglide', 'fish'}, 'gas': {'cow farts', 'unicorn farts', 'elon musk'}})}))

In [10]:
table_params.rename_columns.netgen

'net_generation_kwh'

In [6]:
tttfr = tfr.TestTableTransformer("test_table_id")
actual = tttfr.apply({"test_table_id": tfr.TEST_INPUT_DF})["test_table_id"]

pd.testing.assert_frame_equal(actual, tfr.EXPECTED_OUTPUT_DF)

KeyError: '_typ'

In [ ]:
actual

In [ ]:
tfr.TEST_INPUT_DF

In [ ]:
table_params.rename_columns